In [32]:
import pandas as pd
import numpy as np
import math

In [3]:
courses = pd.read_csv('robotec_courses.csv', index_col='Unnamed: 0')

In [17]:
courses

,group_name,grade_group,name,class_duration,price,classes_per_week,semester_duration_in_weeks
0,a1,0.0,we_do_1.0,2.5,32000.0,1,32
1,a2,0.0,we_do_1.0,2.5,32000.0,1,32
2,ab1,0.5,we_do_2.0,2.5,34000.0,1,32
3,ab2,0.5,we_do_2.0,2.5,34000.0,1,32
4,b1,1.0,we_do_2.0,2.5,38000.0,1,32
5,b2,1.0,we_do_2.0,2.5,38000.0,1,32
6,b3,2.0,we_do_2.0,2.5,38000.0,1,32
7,b4,2.0,scratch_3.0,3.0,42000.0,1,32
8,c1,3.0,scratch_3.0,3.0,42000.0,1,32
9,c2,3.0,ev3_classroom,3.0,42000.0,1,32


I also prepared little sum up of all hardware used during courses. Evey tool price was gotten from open sources. Additional hardware is laptops that are used to write code for our hardware, price was randomly set at $700 value. All price were converted from USD to KZT using 440 USD/KZT rate.

It was also mentioned that every hardware piece can be used simultaneously by 2 students, which is beneficial in terms of equipment payback period.

In [38]:
hardware = pd.read_csv('robotec_hardware.csv', index_col='Unnamed: 0')

In [39]:
hardware

,name,price,additional_hardware_price,students_per_hardware_piece
0,we_do_1.0,88000.0,308000.0,2
1,we_do_2.0,99000.0,308000.0,2
2,scratch_3.0,9680.0,308000.0,2
3,ev3_classroom,204600.0,308000.0,2
4,ev3_mindstorms,204600.0,308000.0,2
5,python,0.0,308000.0,2
6,arduino,12320.0,308000.0,2


In [40]:
hardware['average_price_by_tool'] = hardware['name'].map(courses.groupby('name').mean()['price'])

In [41]:
hardware

,name,price,additional_hardware_price,students_per_hardware_piece,average_price_by_tool
0,we_do_1.0,88000.0,308000.0,2,32000.0
1,we_do_2.0,99000.0,308000.0,2,36400.0
2,scratch_3.0,9680.0,308000.0,2,42000.0
3,ev3_classroom,204600.0,308000.0,2,42000.0
4,ev3_mindstorms,204600.0,308000.0,2,40000.0
5,python,0.0,308000.0,2,42000.0
6,arduino,12320.0,308000.0,2,38000.0


In [47]:
hardware['weeks_to_sell_to '] = round(hardware['price'] / (hardware['students_per_hardware_piece'] * hardware['average_price_by_tool']) * 4, 1)

In [48]:
hardware

,name,price,additional_hardware_price,students_per_hardware_piece,average_price_by_tool,payback_period_in_weeks
0,we_do_1.0,88000.0,308000.0,2,32000.0,5.5
1,we_do_2.0,99000.0,308000.0,2,36400.0,5.4
2,scratch_3.0,9680.0,308000.0,2,42000.0,0.5
3,ev3_classroom,204600.0,308000.0,2,42000.0,9.7
4,ev3_mindstorms,204600.0,308000.0,2,40000.0,10.2
5,python,0.0,308000.0,2,42000.0,0.0
6,arduino,12320.0,308000.0,2,38000.0,0.6


I did 1st task superficialy 😅

## Part 2

To calculate UE we can use 2 approaches: calculating net profit margin **OR** by calculating UE using formula LTV/CAC.

I took values of:
- 400,000 KZT spent on marketing in 2022
- 300 customers in 2022
- Average check of 41,000 KZT
- Salary of teacher per hour 3,000 KZT
- Price of electricity per hour 7.05 KZT
- Lifetime of client of 5 months
- Repeat purchases during period by 100 customers

In [54]:
def cac(m_c, b):
    return round(m_c / b, 1)

In [63]:
def ltv(avp, vc, rpr, lftm):
    return (avp - (vc * avp)) * rpr * lftm

In [109]:
def vc(salary_per_hour, electricity_consumption_per_hour, average_product_price_per_hour):
    electricity_price = 7.05
    return (salary_per_hour + electricity_consumption_per_hour * electricity_price) / average_product_price_per_hour

In [61]:
def rpr(b, r_b):
    return r_b / b

In [115]:
my_cac = cac(400000, 300)

In [135]:
# to calculate average product price per hour i got "average_price_of_course / (average_class_duration) * 4"

my_ltv = ltv(41000, vc(3000, 1.5, (round(courses['price'].mean(),1) / (round(courses['class_duration'].mean(),1) * 4))), rpr(300, 100), 5)

In [136]:
my_ltv / my_cac

6.300306554078942

In [137]:
my_ltv

8400.198728553452

In [138]:
my_cac

1333.3